**PROYECTO:** Precios de vivienda con Técnicas avanzadas de regresión.

**Objetivo:** Predecir `SalePrice` usando técnicas típicas de competición: EDA, limpieza, feature engineering, selección de modelos, validación cruzada y creación de la submission.

**Recomendaciones previas**
- Ejecutar en un entorno con las librerías instaladas (`pandas`, `numpy`, `matplotlib`, `seaborn`, `scikit-learn`, `xgboost`/`lightgbm` opcional).
- Poner `train.csv` y `test.csv` en el mismo folder o usar Kaggle API.

# 1) Instalación (ejecutar si falta algo)

In [ ]:
# !pip install pandas numpy matplotlib seaborn scikit-learn xgboost lightgbm shap --quiet

# 2) Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from scipy.stats import skew
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
import joblib
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
RANDOM_SEED = 42

# 3) Cargar datos
- Coloca `train.csv` y `test.csv` en el directorio o descárgalos con Kaggle API.

In [ ]:
# Si estás en Kaggle o ya tienes los archivos
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()

# 4) Análisis exploratorio básico (EDA)
- Ver las primeras filas, tipos, nulos y distribución de la variable objetivo.

In [ ]:
# Información y resumen
train.info()
train.describe().T

# Distribución objetivo
sns.histplot(train['SalePrice'], kde=True)
plt.title('Distribución de SalePrice (bruta)')
plt.show()

# Prueba de sesgo (skew)
print("Skew:", train['SalePrice'].skew())

Si la variable objetivo está sesgada (típico en precios), aplicamos `log1p` para estabilizar.

In [ ]:
# Variable objetivo transformada
y = np.log1p(train['SalePrice'])
sns.histplot(y, kde=True)
plt.title('Distribución de log1p(SalePrice)')
plt.show()

# Valores faltantes (train)
missing = train.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing.head(30)

# 5) Preparación: unir train + test para procesado conjunto
Esto evita inconsistencias al codificar / escalar.


In [ ]:
train_ID = train['Id']
test_ID = test['Id']

# Guardamos y removemos
train.drop(['Id', 'SalePrice'], axis=1, inplace=True)
test.drop(['Id'], axis=1, inplace=True)

ntrain = train.shape[0]
ntest = test.shape[0]
all_data = pd.concat([train, test], axis=0).reset_index(drop=True)
print("all_data shape:", all_data.shape)

# 6) Limpieza e imputación específica (ejemplos orientados a House Prices)
- Aquí uso reglas comunes (basadas en la naturaleza del dataset). Ajusta según el dataset que uses.

In [ ]:
# Ejemplos de imputación comunes en House Prices
# 1) Columnas donde NA significa "None"
cols_none = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu',
             'GarageType','GarageFinish','GarageQual','GarageCond',
             'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
             'MasVnrType']
for col in cols_none:
    if col in all_data.columns:
        all_data[col] = all_data[col].fillna('None')

# 2) Columnas numéricas donde NA significa 0 (áreas, baños del sótano, etc.)
cols_zero = ['GarageYrBlt','GarageArea','GarageCars','BsmtFinSF1','BsmtFinSF2',
             'BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath','MasVnrArea']
for col in cols_zero:
    if col in all_data.columns:
        all_data[col] = all_data[col].fillna(0)

# 3) Rellenar por moda para algunas categóricas
for col in ['Electrical','MSZoning','KitchenQual','Exterior1st','Exterior2nd','SaleType','Functional']:
    if col in all_data.columns:
        all_data[col] = all_data[col].fillna(all_data[col].mode()[0])

# 4) LotFrontage: imputar por mediana por vecindario (Neighborhood)
if 'LotFrontage' in all_data.columns:
    all_data['LotFrontage'] = all_data.groupby('Neighborhood')['LotFrontage'] \
                                     .transform(lambda x: x.fillna(x.median()))
    # Si aún queda NA
    all_data['LotFrontage'] = all_data['LotFrontage'].fillna(all_data['LotFrontage'].median())


# Verificar nulos restantes
all_data.isnull().sum()[all_data.isnull().sum() > 0].sort_values(ascending=False)

# 7) Feature engineering (típico)
- Crear features útiles: años desde construccion, total baños, etc.

In [ ]:
# Ejemplos
if 'YearBuilt' in all_data.columns and 'YrSold' in all_data.columns:
    all_data['HouseAge'] = all_data['YrSold'] - all_data['YearBuilt']

# Total full & half baths as a single feature
if set(['FullBath','HalfBath','BsmtFullBath','BsmtHalfBath']).issubset(all_data.columns):
    all_data['TotalBath'] = (all_data['FullBath'] + 0.5*all_data['HalfBath'] +
                             all_data['BsmtFullBath'] + 0.5*all_data['BsmtHalfBath'])

# Total square footage
if set(['TotalBsmtSF','1stFlrSF','2ndFlrSF']).issubset(all_data.columns):
    all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

# 8) Tratamiento de variables categóricas ordinales
Algunas características (calidades) son ordinales: mapearlas a números.

In [ ]:
qual_map = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'None':0}
qual_cols = ['ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC','KitchenQual',
             'FireplaceQu','GarageQual','GarageCond','PoolQC']
for col in qual_cols:
    if col in all_data.columns:
        all_data[col] = all_data[col].map(qual_map).fillna(0)

# 9) One-hot encoding para categóricas restantes

In [ ]:
# Identificar columnas categóricas
categorical_cols = all_data.select_dtypes(include=['object']).columns
print("Categorical columns:", len(categorical_cols))

# One-hot encoding (pd.get_dummies)
all_data = pd.get_dummies(all_data, columns=categorical_cols, drop_first=True)
print("Shape after dummies:", all_data.shape)

# 10) Corregir skew en variables numéricas
Aplicar log1p a features numéricos muy sesgados.


In [ ]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew': skewed_feats})
skewed_features = skewness[abs(skewness['Skew']) > 0.75].index

for feat in skewed_features:
    all_data[feat] = np.log1p(all_data[feat])

# 11) Separar de nuevo train / test y preparar X e y


In [ ]:
# Split
X = all_data[:ntrain]
X_test = all_data[ntrain:]
print("X shape:", X.shape, "X_test shape:", X_test.shape)

# Target (transformado)
y = np.log1p(train['SalePrice'])  # record: train variable anterior

# 12) Validación: función RMSE CV

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

def rmse_cv(model, X, y):
    kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kf))
    return rmse

# 13) Modelos base: Lasso, RandomForest, GradientBoosting, XGBoost (si está)

In [ ]:
# Lasso
lasso = make_pipeline(StandardScaler(), Lasso(alpha=0.0005, random_state=RANDOM_SEED))

# RandomForest
rf = RandomForestRegressor(n_estimators=300, random_state=RANDOM_SEED, n_jobs=-1)

# Gradient Boosting
gbr = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=4, random_state=RANDOM_SEED)

# XGBoost (opcional)
try:
    import xgboost as xgb
    xgb_model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=3, subsample=0.8,
                                 colsample_bytree=0.8, random_state=RANDOM_SEED, n_jobs=-1)
    has_xgb = True
except Exception as e:
    print("XGBoost no está instalado o dio error:", e)
    xgb_model = None
    has_xgb = False

# Validación de modelos
models = [('Lasso', lasso), ('RandomForest', rf), ('GradientBoosting', gbr)]
if has_xgb:
    models.append(('XGBoost', xgb_model))

results = []
for name, model in models:
    score = rmse_cv(model, X, y)
    results.append((name, score.mean(), score.std()))
    print(f"{name} RMSE: {score.mean():.5f} (std {score.std():.5f})")

results_df = pd.DataFrame(results, columns=['Model','RMSE_mean','RMSE_std']).sort_values('RMSE_mean')
results_df

# 14) Mejoras: búsqueda rápida de hiperparámetros (ejemplo Lasso o XGBoost)
- Por economía de tiempo, uso RandomizedSearch para XGBoost si está disponible.

In [ ]:
# Ejemplo: búsqueda para XGBoost (rápida)
if has_xgb:
    param_dist = {
        'n_estimators': [500, 800, 1000],
        'learning_rate': [0.01, 0.03, 0.05],
        'max_depth': [3,4,5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0]
    }
    rs = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=10,
                            scoring='neg_mean_squared_error', cv=3, random_state=RANDOM_SEED, n_jobs=-1)
    rs.fit(X, y)
    print("Best params XGB:", rs.best_params_)
    best_xgb = rs.best_estimator_
else:
    best_xgb = None

# 15) Ensamblado simple: promedio (blending) o stacking
- Aquí hago un stacking ligero usando sklearn StackingRegressor (base: Lasso + GBR + XGB si hay).

In [ ]:
estimators = [('lasso', lasso), ('gbr', gbr)]
if has_xgb:
    estimators.append(('xgb', xgb_model))

stack = StackingRegressor(estimators=estimators, final_estimator=Ridge(), cv=5, n_jobs=-1)
stack_score = rmse_cv(stack, X, y)
print("Stacking RMSE:", stack_score.mean(), "std:", stack_score.std())

# 16) Ajuste final y predicción sobre test
- Ajustar el mejor modelo (o combinación) sobre todo el train y predecir.

In [ ]:
# Selecciona modelo final (ejemplo: stack si va bien, si no usa la mejor según CV)
final_model = stack
final_model.fit(X, y)
preds_log = final_model.predict(X_test)
# revertir log(1+x)
preds = np.expm1(preds_log)

# Crear archivo submission
submission = pd.DataFrame({'Id': test_ID, 'SalePrice': preds})
submission.to_csv('submission.csv', index=False)
print("Submission creada: submission.csv (ejemplo, revisa los primeros registros)")
submission.head()

# 17) Guardar modelo

In [ ]:
joblib.dump(final_model, "final_model_stack.pkl")
print("Modelo guardado en final_model_stack.pkl")

# 18) Ideas para mejorar (lista de verificación)
- Feature engineering más profundo (interacciones, polinomios)
- Uso de LightGBM, CatBoost y tuning con Bayesian optimization (Optuna)
- Stacking avanzado (blending con out-of-fold predictions)
- Selección/regularización de features (RFE / L1)
- Análisis de errores: casos con mayor error, outliers
- Usar validación por tiempo o por grupos (si aplica)
- Añadir variables externas (censo, clima, ubicación)

# 19) Consejos para competiciones Kaggle
- Siempre valida localmente con CV antes de subir.
- Usa seed constante para reproducibilidad.
- Revisa que `Id` en la submission corresponda al de `test.csv`.
- Sube varias soluciones (blends) y compara en Public/Private leaderboard.
- Documenta tu pipeline para replicarlo en entrevistas / portafolio.